# Imports

In [1]:
import math
import struct
import numpy as np
import matplotlib.pyplot as plt


# This is the normal cordic implementation

In [2]:
angles_degrees = [45.0,        # 0.785398
                  26.56505118, # 0.46364760905065
                  14.03624347, # 0.24497866316245
                  7.125016349, # 0.1243549945481669
                  3.576334375, # 0.062418809995849
                  1.789910608, # 0.031239833425896
                  0.89517371,  # 0.01562372861675
                  0.447614171, # 0.0078123410625155
                  0.2239105,   # 0.003907975454777
                  0.111905677, # 0.0019531225153184
                  0.055952892, # 0.00097656219141037
                  0.027976453, # 0.00048828121787823
                  0.013988227, # 0.00024414061766576
                  0.006994114, # 0.000122070317559528
                  0.003497057, # 6.103515877976e-5
                  0.001748528  # 3.051757066324e-5
                  ]



def scale_input(angle):
    # Ensure angle is within the range [-pi, pi)
    angle = angle % (2 * math.pi)
    # Ensure angle is positive
    if angle < 0:
        angle += 2 * math.pi
    
    # Determine the number of pi/2 rotations needed to bring the angle to the first quadrant
    num_rotations = math.floor(angle / (0.5 * math.pi))
    
    # Compute the scaled angle in the first quadrant
    scaled_angle = angle - num_rotations * (0.5 * math.pi)
    
    # Determine the sign based on the number of rotations
    sign = 1 if num_rotations % 2 == 0 else - 1
    
    return scaled_angle, sign

def CORDIC_TEST(angle_deg):
      
    #a test of cordic
    print(f"actual Value: {math.cos(math.radians(angle_deg))}\n")
    
    angle = math.radians(angle_deg)
    angle, sign = scale_input(angle)
    angles_radians = [math.radians(i) for i in angles_degrees]
    
    z = 0.0                     # angle
    x = 1.0 / 1.646760258121    # x
    y = 0.0                     # y
    #fix 7 iterations
    for i in range(8):
        print(f"z: {math.degrees(z)}, x: {x}, y: {y}")
        if (z < angle):
            print("too small")
            z = z + angles_radians[i]
            x_plus = x - (y * 2 ** (-i))
            y_plus = y + (x * 2 ** (-i))
            
        elif (z > angle):
            print("too big")
            z = z - angles_radians[i]
            x_plus = x + (y * 2 ** (-i))
            y_plus = y - (x * 2 ** (-i))
            
        else:
            print(f"Exact, Predicted Value: z = {math.degrees(z)} with cos(x) = {x}")
            return
        

        x = x_plus
        y = y_plus
        
    print(f"\nPredicted Value: z = {math.degrees(z)} with cos(x) = {x}, with actual value of predicted: {math.cos(z)}")
    print(f"The result will have a sign of: {sign}")
    
    

CORDIC_TEST(65.0)





actual Value: 0.42261826174069944

z: 0.0, x: 0.6072529350089055, y: 0.0
too small
z: 45.0, x: 0.6072529350089055, y: 0.6072529350089055
too small
z: 71.56505118000001, x: 0.30362646750445277, y: 0.9108794025133583
too big
z: 57.52880771, x: 0.5313463181327923, y: 0.8349727856372451
too small
z: 64.653824059, x: 0.4269747199281367, y: 0.9013910754038441
too small
z: 68.23015843400002, x: 0.3706377777153964, y: 0.9280769953993526
too big
z: 66.44024782600002, x: 0.3996401838216262, y: 0.9164945648457465
too big
z: 65.54507411600001, x: 0.41396041139734097, y: 0.9102501869735337
too big

Predicted Value: z = 65.09745994500003 with cos(x) = 0.4210717409830717, with actual value of predicted: 0.42107602432189006
The result will have a sign of: 1


# Changable Parameters for my Implementation

In [21]:
INT_BITS = 8
FRAC_BITS = 4
NUM_CORDIC_LOOPS = 8



# my new cordic

In [23]:
def float_to_fixed(number, IB=INT_BITS, FB=FRAC_BITS):
    return int(number * (10 ** FB))

def fixed_to_float(fixed, IB=INT_BITS, FB=FRAC_BITS):
    # Calculate the fractional multiplier
    multiplier = 1.0 / (2 ** FB)
    
    # Convert the fixed-point number to floating-point by dividing it by the multiplier
    floating_point = fixed * multiplier
    
    return floating_point


def my_cordic(target_input, IB=INT_BITS, FB=FRAC_BITS):
    
    angles_radians = [float_to_fixed(math.radians(i)) for i in angles_degrees]
    angle = 0
    x = float_to_fixed(1.0 / 1.646760258121)
    y = 0 
    target = float_to_fixed(target_input)

    for i in range(NUM_CORDIC_LOOPS):
        if (target == angle):
            print(f"early termination, {x}")
            return x
        else:
            if (target > angle):
                
                angle = angle + angles_radians[i]
                x_plus = x - (y * 2 ** (-i))
                y_plus = y + (x * 2 ** (-i))
                x = x_plus
                y = y_plus
            else:
                angle = angle + angles_radians[i]
                x_plus = x - (y * 2 ** (-i))
                y_plus = y + (x * 2 ** (-i))
                x = x_plus
                y = y_plus
    return x
    
    
print(float_to_fixed(999.23425234121231))
print(float_to_fixed(-979.54234215234))
print(float_to_fixed(0))
print(float_to_fixed(float(0.234298123811237123)))
print("\n\n")

print(math.cos(45))
print(my_cordic(45))

9992342
-9795423
0
2342



0.5253219888177297
-1639.185567945242
